# Imports Block

In [ ]:
import sqlite3 as sl
import pandas as pd
import numpy as np

import calendar

import plotly.graph_objects as go
from plotly import express as px
from plotly.subplots import make_subplots

from sklearn.linear_model import LinearRegression

# Functions Block

In [ ]:
def query_climate_database(conn, 
                           country, 
                           year_begin, 
                           year_end, 
                           month):
    
    '''
    Get pd.DataFrame from connected Climte Database by specified country (string value), 
    years ineterval and month (separate integer values). 
    
    The table in return contains columns LATITUDE, LONGITUDE, NAME, Country, Year, Month and Temp. 
    
    NOTE: 
    
    The database from the connection must contain following structure of tables and columns 
    
    stations table with at least next columns 
        LATITUDE
        LONGITUDE
        NAME (of station)
        ID
        FIPS 10-4
    countries table with at least next columns 
        Name (of country)
        FIPS 10-4
    temperatures table with at least next columns 
        ID (of station)
        Year
        Month
        Temp
    
    '''
    
    query_string = (
    
    'SELECT s.LATITUDE, s.LONGITUDE, s.NAME, c.Name AS Country, t.Year, t.Month, t.Temp '
    'FROM stations s '
    'LEFT JOIN countries c ON s.[FIPS 10-4] = c.[FIPS 10-4] '
    'LEFT JOIN temperatures t ON s.ID = t.ID '
    f'WHERE c.Name = "{country}" AND (t.Year >= {year_begin} AND t.Year <= {year_end}) AND t.Month = {month}'
    
    )
    
    extracted_data = pd.read_sql_query(query_string, conn)
    
    return extracted_data

def temperature_coefficient_plot(conn, 
                                 country, 
                                 year_begin, 
                                 year_end, 
                                 month, 
                                 min_obs=10, 
                                 **kwargs):
    
    '''
    Builds a graph based on pd.DataFrame from query_climate_database function with specifying minimum observaions 
    (integer value) to filter source data. 
    
    **kwargs specify arguments for px.scatter_mapbox function used to build a graph.
    
    '''
    
    def get_k_from_linreg(sample):
        
        '''
        Fits and applies linear regression to sample (frame) of data. 
        
        '''
        
        linreg = LinearRegression()
        linreg.fit(np.array(sample['Year']).reshape(-1, 1), sample['Temp'])
        increasing_temp_rate = linreg.coef_[0] # k-value from kx+b mathematical function
        
        return increasing_temp_rate
    
    # get source data
    data = query_climate_database(conn, country, year_begin, year_end, month)
    
    # count number of years of data for any given station 
    # and save it in 'obs' (observations) column
    data['obs'] = data.groupby('NAME')['Year'].transform('nunique')
    
    # filter source data by 'min_obs' argument
    filtered_data = data[data['obs'] >= min_obs].reset_index(drop=True, inplace=False)
    
    # apply 'get_k_from_linreg' function to every group (sample) from grouped data with filter 
    # to get an increasing coefficient for each row in this data
    filtered_data = filtered_data.merge(filtered_data.groupby(['NAME', 'Month']
                                                              ).apply(get_k_from_linreg).reset_index(), 
                                        how='left', 
                                        on=['NAME', 'Month'])
    
    # change default name of column got from pd.DataFrame merge method 
    # specifying what it actually means
    filtered_data.rename(columns={0: 'Estimated Yearly Increase (°C)'}, 
                         inplace=True)
    
    # build an object with graph to return it
    fig = px.scatter_mapbox(filtered_data, 
                            lat='LATITUDE', 
                            lon='LONGITUDE', 
                            hover_name='NAME', 
                            hover_data={'Estimated Yearly Increase (°C)': ':.4f'}, 
                            color='Estimated Yearly Increase (°C)',
                            color_continuous_midpoint=0,
                            title=(f'Estimates of yearly increase in temperature in {calendar.month_name[month]} '
                                   f'for stantions in {country}, years {year_begin}-{year_end}'), 
                            **kwargs)
    
    return fig

In [ ]:
def grouped_query_climate_database(conn, 
                                   year_begin, 
                                   year_end, 
                                   month):
    
    '''
    Get pd.DataFrame from connected Climte Database by specified years ineterval and month 
    (separate integer values) grouped by country and year with average temperatures of all months and stations. 
    
    The table in return contains columns Country, Year, Month and [Stations AVG Temp]. 
    
    NOTE: 
    
    The database from the connection must contain following structure of tables and columns 
    
    stations table with at least next columns 
        ID
        FIPS 10-4
    countries table with at least next columns 
        Name (of country)
        FIPS 10-4
    temperatures table with at least next columns 
        ID (of station)
        Year
        Month
        Temp
    
    '''
    
    query_string = (
    
    'SELECT c.Name AS Country, t.Year, t.Month, AVG(t.Temp) AS [Stations AVG Temp]'
    'FROM stations s '
    'LEFT JOIN countries c ON s.[FIPS 10-4] = c.[FIPS 10-4] '
    'LEFT JOIN temperatures t ON s.ID = t.ID '
    f'WHERE (t.Year >= {year_begin} AND t.Year <= {year_end}) AND t.Month = {month} '
    'GROUP BY Country, t.Year'
    
    )
    
    extracted_data = pd.read_sql_query(query_string, conn)
    extracted_data.dropna(subset=['Country'], inplace=True) # sometimes country is None because of bads on left join
    
    return extracted_data

def temperatures_rating_bar(conn, 
                            year_begin, 
                            year_end, 
                            month):
    
    '''
    Builds a bar chart based on pd.DataFrame from grouped_query_climate_database function.
    
    '''
    
    # get source data
    countries_temps_frame = grouped_query_climate_database(conn, year_begin, year_end, month)
    
    # group source data by countries to get its average temperature values all over the specified period 
    # and stations
    countries_avg_temps_frame = countries_temps_frame.groupby('Country', as_index=False)['Stations AVG Temp'].mean()
    
    # 'Stations AVG Temp' now contains only one unique averaged value for every unique country
    countries_avg_temps_frame.rename(columns={'Stations AVG Temp': 'AVG Temp'}, inplace=True)
    
    # define pd.DataFrame with top-10 coldest countries 
    coldest_countries_temps_frame = countries_avg_temps_frame.sort_values(by='AVG Temp', 
                                                                          ignore_index=True, 
                                                                          ascending=True).head(10)
    # define pd.DataFrame with top-10 hottest countries 
    hottest_countries_temps_frame = countries_avg_temps_frame.sort_values(by='AVG Temp', 
                                                                          ignore_index=True, 
                                                                          ascending=False).head(10)
    # (in specified month, averaged by all stations and specified years)
    
    # build an object with graph to return it
    
    fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.25, x_title='°C')
    
    fig.add_trace(
        go.Bar(x=coldest_countries_temps_frame['AVG Temp'], 
               text=coldest_countries_temps_frame['AVG Temp'].round(2), 
               y=coldest_countries_temps_frame['Country'], 
               orientation='h', 
               name='THE COLDEST'
        ), 
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(x=hottest_countries_temps_frame['AVG Temp'], 
               text=hottest_countries_temps_frame['AVG Temp'].round(2), 
               y=hottest_countries_temps_frame['Country'], 
               orientation='h', 
               name='THE HOTTEST'
        ), 
        row=1, col=2
    )
    
    fig.update_yaxes(row=1, col=1, autorange='reversed')
    fig.update_yaxes(row=1, col=2, autorange='reversed')
    
    fig.update_layout(title={'text': ('Top-10 countries by average temperature in '
                                      f'{calendar.month_name[month]}, years '
                                      f'{year_begin}-{year_end}'), 
                             'y': 0.9,
                             'x': 0.45,
                             'xanchor': 'center',
                             'yanchor': 'top'}, 
                      paper_bgcolor='rgba(0,0,0,0)', 
                      plot_bgcolor='rgba(0,0,0,0)'
    )
    
    return fig

In [ ]:
def countries_query_climate_database(conn, 
                                     countries_list, 
                                     year, 
                                     func_for_temp='AVG'):
    
    '''
    Get pd.DataFrame from connected Climte Database by specified countries (string values in list) 
    and year (integer value) with optional 'func_for_temp' argument (one of SQL functions for columns 
    like 'AVG', 'MAX', 'MIN' etc.) grouped by country, year and month with average temperatures of all stations. 
    
    The table in return contains columns Country, Year, Month and [Stations AVG Temp]. 
    
    NOTE: 
    
    The database from the connection must contain following structure of tables and columns 
    
    stations table with at least next columns 
        ID
        FIPS 10-4
    countries table with at least next columns 
        Name (of country)
        FIPS 10-4
    temperatures table with at least next columns 
        ID (of station)
        Year
        Month
        Temp
    
    '''
    
    # write values from countries_list into one string
    countries_string = ', '.join([f'"{country}"' for country in countries_list])
    
    query_string = (
    
    f'SELECT c.Name AS Country, t.Year, t.Month, {func_for_temp}(t.Temp) AS [Stations AVG Temp]'
    'FROM stations s '
    'LEFT JOIN countries c ON s.[FIPS 10-4] = c.[FIPS 10-4] '
    'LEFT JOIN temperatures t ON s.ID = t.ID '
    f'WHERE t.Year = {year} AND Country in ({countries_string})'
    'GROUP BY Country, t.Year, t.Month'
    
    )
    
    extracted_data = pd.read_sql_query(query_string, conn)
    
    return extracted_data

def temperature_dynamics_scatter(conn, 
                                 countries_list, 
                                 year, 
                                 func_for_temp='AVG'):
    
    '''
    Builds a scatter based on pd.DataFrame from countries_query_climate_database function.
    
    '''
    
    # get source data
    data = countries_query_climate_database(conn, countries_list, year, func_for_temp)
    
    # build an object with graph to return it
    
    fig = go.Figure()
    
    for country in countries_list:
        
        country_data = data[data['Country'] == country].sort_values(by='Month', ascending=True, ignore_index=True)
        country_data['Month'] = country_data['Month'].apply(lambda month: calendar.month_name[month])
        
        fig.add_trace(
            go.Scatter(x=country_data['Month'], 
                       y=country_data['Stations AVG Temp'].round(2), 
                       mode="lines",
                       name=country
            )
        )
    
    fig.update_traces(textposition="bottom right")
    
    fig.layout.yaxis.ticksuffix = ' °C '
    
    fig.update_layout(title=f'{func_for_temp.upper()} temperature dynamics in {year}', 
                      paper_bgcolor='rgba(0,0,0,0)', 
                      plot_bgcolor='rgba(0,0,0,0)')
    
    return fig

# Database Creation

In [ ]:
# check or create database file
climate_db_conn = sl.connect('climate.db')

# upload local csv tables
temperatures_frame = pd.read_csv("temps_stacked.csv")
stations_frame = pd.read_csv("station-metadata.csv")
countries_frame = pd.read_csv("countries.csv")

# create 'FIPS 10-4' column from related 'ID' values in table of stations
stations_frame['FIPS 10-4'] = [row[:2] for row in stations_frame['ID']]

# submit the tables into the database
temperatures_frame.to_sql('temperatures', climate_db_conn, if_exists='replace', index=False)
stations_frame.to_sql('stations', climate_db_conn, if_exists='replace', index=False)
countries_frame.to_sql('countries', climate_db_conn, if_exists='replace', index=False)

# here and below don't forget to close connection to the database
climate_db_conn.close()

# Query Function Example

In [ ]:
climate_db_conn = sl.connect('climate.db') # here and below don't forget to open connection to the database
India_climate_sample = query_climate_database(conn=climate_db_conn, 
                                              country='India', 
                                              year_begin=1980, 
                                              year_end=2020, 
                                              month=1)
climate_db_conn.close()
display(India_climate_sample)

# Temperatures Plot Examples

In [ ]:
climate_db_conn = sl.connect('climate.db')
color_map = px.colors.diverging.Spectral_r
fig1 = temperature_coefficient_plot(conn=climate_db_conn, 
                                    country="India", 
                                    year_begin=1980, 
                                    year_end=2020, 
                                    month=1, 
                                    min_obs=10, 
                                    zoom=2,
                                    mapbox_style="carto-positron",
                                    color_continuous_scale=color_map)
climate_db_conn.close()
fig1.show()

In [ ]:
climate_db_conn = sl.connect('climate.db')
color_map = px.colors.diverging.curl
fig2 = temperature_coefficient_plot(conn=climate_db_conn, 
                                    country="United States", 
                                    year_begin=2000, 
                                    year_end=2021, 
                                    month=11, 
                                    min_obs=21, 
                                    zoom=3,
                                    mapbox_style="carto-positron",
                                    color_continuous_scale=color_map)
climate_db_conn.close()
fig2.show()

# More Plot Examples

In [ ]:
climate_db_conn = sl.connect('climate.db')
bar_fig = temperatures_rating_bar(conn=climate_db_conn, 
                                  year_begin=1991, 
                                  year_end=2021, 
                                  month=11)
climate_db_conn.close()
bar_fig.show()

In [ ]:
climate_db_conn = sl.connect('climate.db')
scatter_fig = temperature_dynamics_scatter(conn=climate_db_conn, 
                                           countries_list=['Antarctica', 'Russia', 'Mongolia', 'Greenland'], 
                                           year=2020, 
                                           func_for_temp='AVG')
climate_db_conn.close()
scatter_fig.show()